## Importing the Data

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# Constants
DATASET_DIR = './data/'
SAVE_DIR = './'

X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel.tsv'), sep='\t', encoding='ISO-8859-1')
y = X['domain1_score']
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [3]:
X.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


Minimum and Maximum Scores for each essay set.

In [4]:
minimum_scores = np.array([-1, 2, 1, 0, 0, 0, 0, 0, 0])
maximum_scores = np.array([-1, 12, 6, 3, 3, 4, 4, 30, 60])

In [5]:
old_min = minimum_scores[X['essay_set']]
old_max = maximum_scores[X['essay_set']]
old_range = old_max - old_min
new_min = 0
new_max = 100
new_range = (new_max - new_min)  
X['score'] = (((X['domain1_score'] - old_min) * new_range) / old_range) + new_min

# round score to nearest integer for cohen kappa calculation
y = np.round(X['score'])

X.head()

,essay_id,essay_set,essay,domain1_score,score
0,1,1,"Dear local newspaper, I think effects computer...",8,60.0
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,70.0
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,50.0
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,80.0
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,60.0


## Preprocessing the Data

We will preprocess all essays and convert them to feature vectors so that they can be fed into the RNN.

These are all helper functions used to clean the essays.

In [6]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tmax\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tmax\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [101]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    for word in words:
        if word in model:
            num_words += 1
            featureVec = np.add(featureVec, model[word])       
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def makeFeatureVec2(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    for word in words:
        if word in model:
            featureVec = np.add(featureVec, model[word])
    if len(words) != 0:
        featureVec = np.divide(featureVec,float(len(words)))
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

def getAvgFeatureVecs2(essay, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    essayFeatureVecs = np.zeros((len(essay),num_features),dtype="float32")
    for cnt, sentence in enumerate(essay):
        essayFeatureVecs[cnt] = makeFeatureVec2(sentence, model, num_features)
    return essayFeatureVecs

In [26]:
X

,essay_id,essay_set,essay,domain1_score,score
0,1,1,"Dear local newspaper, I think effects computer...",8,60.000000
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,70.000000
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,50.000000
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,80.000000
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,60.000000
...,...,...,...,...,...
12971,21626,8,In most stories mothers and daughters are eit...,35,58.333333
12972,21628,8,I never understood the meaning laughter is th...,32,53.333333
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",40,66.666667
12974,21630,8,Trippin' on fen...,40,66.666667


In [120]:
cv2 = KFold(n_splits=5, shuffle=True)
trainData = []
testData = []
y_trainData = []
y_testData = []
for traincv, testcv in cv2.split(X):
    print('##Fold Started')
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    
    y_trainData.append(y_train)
    y_testData.append(y_test)
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']

    trainDataVecs = []
    testDataVecs = []
    
    for essay in train_essays:
        # Obtaining all sentences from the training essays.
        sentences = essay_to_sentences(essay, remove_stopwords = True)
        trainDataVec = getAvgFeatureVecs2(sentences, embedding_dict, num_features)
        trainDataVecs.append(np.array(trainDataVec))

    for essay in test_essays:
        # Obtaining all sentences from the training essays.
        sentences = essay_to_sentences(essay, remove_stopwords = True)
        testDataVec = getAvgFeatureVecs2(sentences, embedding_dict, num_features)
        testDataVecs.append(np.array(testDataVec))
    
    trainDataVecs = pad_sequences(trainDataVecs, maxlen=128, padding='pre')
    testDataVecs = pad_sequences(testDataVecs, maxlen=128, padding='pre')
    
    trainData.append(np.array(trainDataVecs))
    testData.append(np.array(testDataVecs))
    print(len(trainDataVecs))
    print(len(testDataVecs))

##Fold Started
10380
2596
##Fold Started
10381
2595
##Fold Started
10381
2595
##Fold Started
10381
2595
##Fold Started
10381
2595


In [121]:
trainData[0].shape

(10380, 128, 200)

In [123]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=5, verbose=0, mode='auto')
sentence_model = get_sentence_model()
sentence_model.fit(trainData[0], y_trainData[0], batch_size=64, epochs=50, callbacks=[early_stopping])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 128, 200)          320800    
_________________________________________________________________
lstm_15 (LSTM)               (None, 64)                67840     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 388,705
Trainable params: 388,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
163/163 [==============================] - 87s 516ms/step - loss: 2744.1730 - mae: 47.4264
Epoch 2/50
163/163 [==============================] - 90s 554ms/step - loss: 1766.1610 - mae: 36.5305
Epoch 3/50
163/163 [======================

In [124]:
y_pred = sentence_model.predict(testData[0])

# Save any one of the 8 models.
if count == 5:
     sentence_model.save('./final_lstm.h5')

# Round y_pred to the nearest integer.
y_pred = np.round(y_pred)

# Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
result = cohen_kappa_score(y_testData[0].values,y_pred,weights='quadratic')
print("Kappa Score: {}".format(result))
results.append(result)

count += 1

Kappa Score: 0.4727047860436322


## Defining the model 

Here we define a 2-Layer LSTM Model. 

Note that instead of using sigmoid activation in the output layer we will use
Relu since we are not normalising training labels.

In [104]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from tensorflow.keras.models import Sequential, load_model, model_from_config
import tensorflow.keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(200, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 200], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

def get_sentence_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(200, dropout=0.4, recurrent_dropout=0.4, input_shape=[128, 200], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [13]:
# load glove embeddings
embedding_dict={}
with open(os.path.join(DATASET_DIR, 'glove/glove.6B.200d.txt'),'r', encoding='UTF8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:],'float32')
        embedding_dict[word] = vectors


## Training Phase

Now we train the model on the dataset.

We will use 5-Fold Cross Validation and measure the Quadratic Weighted Kappa for each fold.
We will then calculate Average Kappa for all the folds.

## Original Trainig Phase

In [14]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

cv = KFold(n_splits=5, shuffle=True)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(X):
    
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
        # Obtaining all sentences from the training essays.
        sentences += essay_to_sentences(essay, remove_stopwords = True)
    
#     # Initializing variables for word2vec model.
    num_features = 200 
#     min_word_count = 40
#     num_workers = 4
#     context = 10
#     downsampling = 1e-3

#     print("Training Word2Vec Model...")
#     model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

#     model.init_sims(replace=True)
#     model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)
    
    model = embedding_dict
    
    # Generate training and testing data word vectors.
    clean_train_essays = []
    for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    y_pred = lstm_model.predict(testDataVecs)
    
    # Save any one of the 8 models.
    if count == 5:
         lstm_model.save('./final_lstm.h5')
            
    # Round y_pred to the nearest integer.
    y_pred = np.round(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1


--------Fold 1--------

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 200)            320800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                67840     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 388,705
Trainable params: 388,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
163/163 [==============================] - 4s 3ms/step - loss: 3123.5160 - mae: 51.0037
Epoch 2/50
163/163 [==============================] - 1s 4ms/step - loss: 2157.0610 - mae: 41.2123
Epoch 3/50
163/163 [=====

163/163 [==============================] - 1s 3ms/step - loss: 461.7545 - mae: 16.8436
Epoch 16/50
163/163 [==============================] - 1s 3ms/step - loss: 450.8210 - mae: 16.6344
Epoch 17/50
163/163 [==============================] - 1s 3ms/step - loss: 438.8937 - mae: 16.3423
Epoch 18/50
163/163 [==============================] - 1s 3ms/step - loss: 447.5435 - mae: 16.5893
Epoch 19/50
163/163 [==============================] - 1s 3ms/step - loss: 428.4290 - mae: 16.3041
Epoch 20/50
163/163 [==============================] - 1s 3ms/step - loss: 418.1288 - mae: 16.0558
Epoch 21/50
163/163 [==============================] - 1s 4ms/step - loss: 421.5110 - mae: 16.0796
Epoch 22/50
163/163 [==============================] - 1s 4ms/step - loss: 418.0497 - mae: 15.9666
Epoch 23/50
163/163 [==============================] - 1s 4ms/step - loss: 420.4483 - mae: 16.0518
Epoch 24/50
163/163 [==============================] - 1s 4ms/step - loss: 413.1053 - mae: 15.9553
Epoch 25/50
163/163 [=

163/163 [==============================] - 1s 4ms/step - loss: 365.9379 - mae: 15.0772
Epoch 39/50
163/163 [==============================] - 1s 4ms/step - loss: 371.8936 - mae: 15.2061
Epoch 40/50
163/163 [==============================] - 1s 4ms/step - loss: 368.4802 - mae: 15.0462
Epoch 41/50
163/163 [==============================] - 1s 4ms/step - loss: 368.4769 - mae: 14.9644
Epoch 42/50
163/163 [==============================] - 1s 4ms/step - loss: 359.7673 - mae: 14.7682
Epoch 43/50
163/163 [==============================] - 1s 4ms/step - loss: 359.1573 - mae: 14.8192
Epoch 44/50
163/163 [==============================] - 1s 4ms/step - loss: 362.7346 - mae: 14.9111
Epoch 45/50
163/163 [==============================] - 1s 4ms/step - loss: 352.1140 - mae: 14.7669
Epoch 46/50
163/163 [==============================] - 1s 4ms/step - loss: 352.2024 - mae: 14.6752
Epoch 47/50
163/163 [==============================] - 1s 4ms/step - loss: 347.5759 - mae: 14.5553
Epoch 48/50
163/163 [=

163/163 [==============================] - 4s 5ms/step - loss: 3063.6701 - mae: 50.5177
Epoch 2/50
163/163 [==============================] - 1s 5ms/step - loss: 2129.2920 - mae: 40.9544
Epoch 3/50
163/163 [==============================] - 1s 4ms/step - loss: 1546.8970 - mae: 33.5919
Epoch 4/50
163/163 [==============================] - 1s 5ms/step - loss: 1126.8598 - mae: 27.8835
Epoch 5/50
163/163 [==============================] - 1s 4ms/step - loss: 844.6284 - mae: 23.7072
Epoch 6/50
163/163 [==============================] - 1s 4ms/step - loss: 669.3601 - mae: 20.9086
Epoch 7/50
163/163 [==============================] - 1s 4ms/step - loss: 598.5200 - mae: 19.5900
Epoch 8/50
163/163 [==============================] - 1s 4ms/step - loss: 591.1925 - mae: 19.4609
Epoch 9/50
163/163 [==============================] - 1s 4ms/step - loss: 542.5459 - mae: 18.5098
Epoch 10/50
163/163 [==============================] - 1s 4ms/step - loss: 499.3449 - mae: 17.6173
Epoch 11/50
163/163 [=====

In [15]:
print("Average Kappa score after a 5-fold cross validation: ", np.round(np.array(results).mean(),decimals=4))

Average Kappa score after a 5-fold cross validation:  0.6427


In [16]:
import math
from gensim.test.utils import datapath

contentBad = """
    In “Let there be dark,” Paul Bogard talks about the importance of darkness.

Darkness is essential to humans. Bogard states, “Our bodies need darkness to produce the hormone melatonin, which keeps certain cancers from developing, and our bodies need darkness for sleep, sleep. Sleep disorders have been linked to diabetes, obesity, cardiovascular disease and depression and recent research suggests are main cause of “short sleep” is “long light.” Whether we work at night or simply take our tablets, notebooks and smartphones to bed, there isn’t a place for this much artificial light in our lives.” (Bogard 2). Here, Bogard talks about the importance of darkness to humans. Humans need darkness to sleep in order to be healthy.

Animals also need darkness. Bogard states, “The rest of the world depends on darkness as well, including nocturnal and crepuscular species of birds, insects, mammals, fish and reptiles. Some examples are well known—the 400 species of birds that migrate at night in North America, the sea turtles that come ashore to lay their eggs—and some are not, such as the bats that save American farmers billions in pest control and the moths that pollinate 80% of the world’s flora. Ecological light pollution is like the bulldozer of the night, wrecking habitat and disrupting ecosystems several billion years in the making. Simply put, without darkness, Earth’s ecology would collapse...” (Bogard 2). Here Bogard explains that animals, too, need darkness to survive.
""" 

contentGood = """
    In response to our world’s growing reliance on artificial light, writer Paul Bogard argues that natural darkness should be preserved in his article “Let There be dark”. He effectively builds his argument by using a personal anecdote, allusions to art and history, and rhetorical questions.

Bogard starts his article off by recounting a personal story – a summer spent on a Minnesota lake where there was “woods so dark that [his] hands disappeared before [his] eyes.” In telling this brief anecdote, Bogard challenges the audience to remember a time where they could fully amass themselves in natural darkness void of artificial light. By drawing in his readers with a personal encounter about night darkness, the author means to establish the potential for beauty, glamour, and awe-inspiring mystery that genuine darkness can possess. He builds his argument for the preservation of natural darkness by reminiscing for his readers a first-hand encounter that proves the “irreplaceable value of darkness.” This anecdote provides a baseline of sorts for readers to find credence with the author’s claims.

Bogard’s argument is also furthered by his use of allusion to art – Van Gogh’s “Starry Night” – and modern history – Paris’ reputation as “The City of Light”. By first referencing “Starry Night”, a painting generally considered to be undoubtedly beautiful, Bogard establishes that the natural magnificence of stars in a dark sky is definite. A world absent of excess artificial light could potentially hold the key to a grand, glorious night sky like Van Gogh’s according to the writer. This urges the readers to weigh the disadvantages of our world consumed by unnatural, vapid lighting. Furthermore, Bogard’s alludes to Paris as “the famed ‘city of light’”. He then goes on to state how Paris has taken steps to exercise more sustainable lighting practices. By doing this, Bogard creates a dichotomy between Paris’ traditionally alluded-to name and the reality of what Paris is becoming – no longer “the city of light”, but moreso “the city of light…before 2 AM”. This furthers his line of argumentation because it shows how steps can be and are being taken to preserve natural darkness. It shows that even a city that is literally famous for being constantly lit can practically address light pollution in a manner that preserves the beauty of both the city itself and the universe as a whole.

Finally, Bogard makes subtle yet efficient use of rhetorical questioning to persuade his audience that natural darkness preservation is essential. He asks the readers to consider “what the vision of the night sky might inspire in each of us, in our children or grandchildren?” in a way that brutally plays to each of our emotions. By asking this question, Bogard draws out heartfelt ponderance from his readers about the affecting power of an untainted night sky. This rhetorical question tugs at the readers’ heartstrings; while the reader may have seen an unobscured night skyline before, the possibility that their child or grandchild will never get the chance sways them to see as Bogard sees. This strategy is definitively an appeal to pathos, forcing the audience to directly face an emotionally-charged inquiry that will surely spur some kind of response. By doing this, Bogard develops his argument, adding gutthral power to the idea that the issue of maintaining natural darkness is relevant and multifaceted.

Writing as a reaction to his disappointment that artificial light has largely permeated the prescence of natural darkness, Paul Bogard argues that we must preserve true, unaffected darkness. He builds this claim by making use of a personal anecdote, allusions, and rhetorical questioning.
"""

def testContent(content):
    if len(content) > 20:
        num_features = 200
        clean_test_essays = []
        clean_test_essays.append(essay_to_wordlist( content, remove_stopwords=True ))
        testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
        testDataVecs = np.array(testDataVecs)
        testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

        preds = lstm_model.predict(testDataVecs)

        if math.isnan(preds):
            preds = 0
        else:
            preds = np.round(preds)

        if preds < 0:
            preds = 0
    else:
        preds = 0

    return preds
    
print("the SAT 1 score essay scored", testContent(contentBad))
print("the SAT 4 score essay scored", testContent(contentGood))

the SAT 1 score essay scored [[63.]]
the SAT 4 score essay scored [[78.]]


In [17]:
import pickle

# Pickle glove embeddings
with open('embeddings.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)